<a href="https://colab.research.google.com/github/paulo-marquesmorgado/EConcepts/blob/main/Assignm2_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Economic Concepts Week 3, Assignment 2

## Macro Economic Analysis

The objective of this assignment is to guide you through a reliable and rich source of macroeconomic data and to
show how much you can learn from it by simply plotting the data and making some straightforward calculations.


In [ ]:
StudentName   = "Klein"
StudentNumber = "1234567"
WEB_Data_Github_location = 'https://github.com/paulo-marquesmorgado/EConcepts/blob/main/SNA_TABLE1_ARCHIVE_ALL-1950-1918.csv'
# Raw URL for the CSV file on GitHub
file_name = "SNA_TABLE1_ARCHIVE_ALL-1950-1918.csv"
Data_Github_location = 'https://raw.githubusercontent.com/paulo-marquesmorgado/EConcepts/main/SNA_TABLE1_ARCHIVE_ALL-1950-1918.csv'
assignment="assignment_2"
myheader = 'Economic Concepts Assignment 2 Report'
mytitle = 'Inventory Investment Analysis'
mysource = 'OECD statistics at stats.oecd.org'

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=022149cc8aa23114706f8c8e1f2048f48328bf00ef0d229159ecf41ca45ddccc
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fpdf import FPDF
import os

## Support Functions to Calculate the statistics

In [ ]:
def calculate_and_display_stats(this_df):
    # Compute statistics
    max_inventory_investment = this_df['Inv_Perc_GDP'].max()
    min_inventory_investment = this_df['Inv_Perc_GDP'].min()
    mean_inventory_investment = this_df['Inv_Perc_GDP'].mean()
    std_dev_inventory_investment = this_df['Inv_Perc_GDP'].std()

    # Display the results
    print(f"Max Inventory Investment as Percentage of GDP: {max_inventory_investment:.2f}")
    print(f"Min Inventory Investment as Percentage of GDP: {min_inventory_investment:.2f}")
    print(f"Standard Deviation of Inventory Investment as Percentage of GDP: {std_dev_inventory_investment:.2f}")
    print("")
    print(f"It's important to consider the context of each year to interpret these values effectively.")
    print(f"For instance, negative percentages may indicate destocking or economic challenges,")
    print(f"while positive percentages could imply inventory buildup or economic expansion.")


### Create the function process_dataframe(df)  
The provided code snippet serves to manipulate a DataFrame called 'df' with the goal of creating a new DataFrame 'df_new' that retains specific columns, compacts the DataFrame's index, and removes duplicate rows based on a particular column.

Then reshape and prepare a DataFrame named 'df_pivot' for analysis by pivoting df, renaming columns, and resetting the index.
Finally, perform data operations on the DataFrames 'df_new' and 'df_pivot,' with the ultimate goal of merging them based on the 'Year' column and calculating the inventory investment as a percentage of GDP.

In [ ]:
def process_dataframe(df):
    # Create a new DataFrame 'df_new'
    selected_columns = ['LOCATION', 'Country', 'Year']
    df_new = df[selected_columns]
    # Reset the index to compact the DataFrame
    df_new.reset_index(drop=True, inplace=True)
    # Drop duplicates based on 'Year'
    df_new = df_new.drop_duplicates(subset='Year', keep='first')

    # Pivot the DataFrame to separate "Inventory" and "GDP" columns
    df_pivot = df.pivot(index='Year', columns='Transaction', values='Value')
    # Rename the columns to match your desired column names
    # df_pivot = df_pivot.rename(columns={'Changes in inventories': 'Inventory', 'Gross domestic product (expenditure approach)': 'GDP'})
    # Reset the index to make "Year" a regular column
    df_pivot.reset_index(inplace=True)

    # Merge the DataFrames df_new and df_pivot on 'Year'
    merged_df = df_new.merge(df_pivot, on=['Year'])
    # Compute inventory investment as a percentage of GDP
    # merged_df['Inv_Perc_GDP'] = (merged_df['Inventory'] / merged_df['GDP']) * 100
    # merged_df['Inv_Perc_GDP'] = (merged_df['Changes in inventories'] / merged_df['Gross domestic product (expenditure approach)']) * 100
    # merged_df = merged_df.dropna(subset=['Inv_Perc_GDP'])
    return merged_df


In [ ]:
def process_dataframe_LocationYear(df):
    # Create a new DataFrame 'df_new'
    selected_columns = ['LOCATION_Year', 'LOCATION', 'Country', 'Year']
    df_new = df[selected_columns]
    # Reset the index to compact the DataFrame
    df_new.reset_index(drop=True, inplace=True)
    # Drop duplicates based on 'Year'
    df_new = df_new.drop_duplicates(subset='LOCATION_Year', keep='first')

    # Pivot the DataFrame to separate "Inventory" and "GDP" columns
    df_pivot = df.pivot(index='LOCATION_Year', columns='Transaction', values='Value')
    # Rename the columns to match your desired column names
    # df_pivot = df_pivot.rename(columns={'Changes in inventories': 'Inventory', 'Gross domestic product (expenditure approach)': 'GDP'})
    # Reset the index to make "Year" a regular column
    df_pivot.reset_index(inplace=True)

    # Merge the DataFrames df_new and df_pivot on 'Year'
    merged_df = df_new.merge(df_pivot, on=['LOCATION_Year'])
    # Compute inventory investment as a percentage of GDP
    # merged_df['Inv_Perc_GDP'] = (merged_df['Inventory'] / merged_df['GDP']) * 100
    # merged_df['Inv_Perc_GDP'] = (merged_df['Changes in inventories'] / merged_df['Gross domestic product (expenditure approach)']) * 100
    # merged_df = merged_df.dropna(subset=['Inv_Perc_GDP'])
    return merged_df



---

## STEP 1: Load the CSV file

In [ ]:
# Step 1: Load the CSV file
original_df = pd.read_csv(Data_Github_location)
# print(original_df.head())

# Create the 'LOCATION_Year' column by concatenating 'LOCATION' and 'Year'
original_df['LOCATION_Year'] = original_df['LOCATION'].astype(str) + '_' + original_df['Year'].astype(str)
# print(original_df)
# original_df['Transaction'].unique()

## STEP 2: Display Graph and Calculations

### Inventory Investment as Percentage of GDP Over Time
(Using the library Seaborn for plotting the graph)

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# Create a dropdown widget to select a country
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')
# Create widgets to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Define a function to update the plot based on the selected country and year range
def update_plot(selected_country, start_year, end_year):
    # Filter the DataFrame based on the selected country and year range
    df = original_df[
        (original_df['Transaction'].isin(['Changes in inventories', 'Gross domestic product (expenditure approach)'])) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    # merged_df = process_dataframe(df)
    merged_df = process_dataframe_LocationYear(df)
    merged_df['Inv_Perc_GDP'] = (merged_df['Changes in inventories'] / merged_df['Gross domestic product (expenditure approach)']) * 100
    merged_df = merged_df.dropna(subset=['Inv_Perc_GDP'])

    # Plot the graph
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=merged_df, x='Year', y='Inv_Perc_GDP', marker='o')
    title = f'Inventory Investment as Percentage of GDP Over Time - {selected_country}'
    plt.title(title)
    plt.xlabel('Year')
    plt.ylabel('Inventory Investment as Percentage of GDP')
    plt.grid(True)
    plt.show()
    # Call the statistics with your 'merged_df' DataFrame
    calculate_and_display_stats(merged_df)

# Define an observer to update the plot when the dropdown value changes
widgets.interactive(update_plot, selected_country=country_dropdown, start_year=start_year_slider, end_year=end_year_slider)


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

### Inventory Investment as Percentage of GDP Over Time
(Using the library Altair for plotting the graph)

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import ipywidgets as widgets
import altair as alt  # Altair for Vega-Lite plotting
from IPython.display import display
import seaborn as sns

# Create a dropdown widget to select a country
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')

# Create widgets to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Define a function to update the plot based on the selected country and year range
def update_plot(selected_country, start_year, end_year):
    # Filter the DataFrame based on the selected country and year range
    df = original_df[
        # (original_df['Transaction'] == selected_transaction) &
        (original_df['Transaction'].isin(['Changes in inventories', 'Gross domestic product (expenditure approach)'])) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe(df)
    merged_df['Inv_Perc_GDP'] = (merged_df['Changes in inventories'] / merged_df['Gross domestic product (expenditure approach)']) * 100
    merged_df = merged_df.dropna(subset=['Inv_Perc_GDP'])

    # Create the Vega-Lite chart
    chart = alt.Chart(merged_df).mark_line(point=True).encode(
        x='Year:O',  # Year as ordinal (categorical)
        y='Inv_Perc_GDP:Q',  # Inv_Perc_GDP as quantitative
        color=alt.value('blue')  # Set color to blue
    ).properties(
        width=800,  # Set chart width
        height=400,  # Set chart height
        title=f'Inventory Investment as Percentage of GDP Over Time - {selected_country}'
        # title=f'{selected_country} ({selected_transaction})'
    )

    # Show the chart
    display(chart)

    # Call the statistics with your 'merged_df' DataFrame
    calculate_and_display_stats(merged_df)

# Define an observer to update the plot when the dropdown value changes
widgets.interactive(update_plot, selected_country=country_dropdown, start_year=start_year_slider, end_year=end_year_slider)



interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

## Plot Graph for a specific Measure and specific Country

In [ ]:
# Import the necessary libraries:
import os
import pandas as pd
import ipywidgets as widgets
import altair as alt
from IPython.display import display
import seaborn as sns

# Create a dropdown widget to select a country:
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')

# Create a dropdown widget to select a transaction:
# transaction_dropdown = widgets.Dropdown(options=original_df['Transaction'].unique(), description='Transaction:')
transaction_dropdown = widgets.Dropdown(options=original_df['Transaction'].unique(), description='Transaction:', style={'description_width': 'initial', 'width': '500px'})

# Create widgets to select start_year and end_year:
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Modify the update_plot function to include the selected_transaction argument and filter the DataFrame based on the selected country, transaction, and year range:
def update_plot(selected_country, selected_transaction, start_year, end_year):
    # Filter the DataFrame based on the selected country, transaction, and year range
    df = original_df[
        (original_df['Transaction'] == selected_transaction) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    # merged_df = process_dataframe(df)

    merged_df = process_dataframe_LocationYear(df)
    # print(merged_df)

    # merged_df['Inv_Perc_GDP'] = (merged_df['Changes in inventories'] / merged_df['Gross domestic product (expenditure approach)']) * 100
    # merged_df = merged_df.dropna(subset=['Inv_Perc_GDP'])

    # Create the Altair chart
    chart = alt.Chart(merged_df).mark_line(point=True).encode(
        x='Year:O',  # Year as ordinal (categorical)
        # y='Inv_Perc_GDP:Q',  # Inv_Perc_GDP as quantitative
        y=alt.Y(selected_transaction),  # selected transaction as quantitative
        color=alt.value('blue')  # Set color to blue
    ).properties(
        width=800,  # Set chart width
        height=400,  # Set chart height
        title=f'{selected_country} - {selected_transaction}'
    )

    # Show the chart
    display(chart)

    # Call the statistics with your 'merged_df' DataFrame
    # calculate_and_display_stats(merged_df)

# Define an observer to update the plot when the dropdown value changes:
widgets.interactive(update_plot, selected_country=country_dropdown, selected_transaction=transaction_dropdown, start_year=start_year_slider, end_year=end_year_slider)


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

## Plot Graph for a specific Measure and specific Country (with a linear line)

In [ ]:
# Import the necessary libraries:
import os
import pandas as pd
import ipywidgets as widgets
import altair as alt
from IPython.display import display
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression

# Create a dropdown widget to select a country:
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')

# Create a dropdown widget to select a transaction:
transaction_dropdown = widgets.Dropdown(options=original_df['Transaction'].unique(), description='Transaction:', style={'description_width': 'initial', 'width': '500px'})

# Create widgets to select start_year and end_year:
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Create a radio button widget to toggle showing the linear curve:
show_linear_curve_radio = widgets.RadioButtons(options=[('No', False), ('Yes', True)], description='Show Linear Curve:', value=False)

# Modify the update_plot function to include the selected_transaction, start_year, end_year, and show_linear_curve arguments:
def update_plot(selected_country, selected_transaction, start_year, end_year, show_linear_curve):
    # Filter the DataFrame based on the selected country, transaction, and year range
    df = original_df[
        (original_df['Transaction'] == selected_transaction) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe_LocationYear(df)

    # Create the linear regression model
    # merged_df1 = add_linear_curve_to_dataframe(processed_df, selected_transaction)
    # Create the linear regression model
    model = LinearRegression()
    X = merged_df['Year'].values.reshape(-1, 1)
    y = merged_df[selected_transaction].values.reshape(-1, 1)
    model.fit(X, y)
    linear_values = model.predict(X)
    # Add the linear regression values to the DataFrame
    merged_df[selected_transaction + '_linear'] = linear_values.flatten()

    # Create the Altair chart for the selected transaction
    chart = alt.Chart(merged_df).mark_line(point=True).encode(
        x='Year:O',  # Year as ordinal (categorical)
        y=alt.Y(selected_transaction),  # selected transaction as quantitative
        color=alt.value('blue')  # Set color to blue
    ).properties(
        width=800,  # Set chart width
        height=400,  # Set chart height
        title=f'{selected_country} - {selected_transaction}'
    )

    # Create the linear curve if the radio button is selected
    if show_linear_curve:
        print("SHOW!")
        linear_data = selected_transaction + '_linear:Q'
        linear_curve = alt.Chart(merged_df).mark_line(strokeDash=[10, 5], color='red').encode(
            x='Year:O',
            y=alt.Y(linear_data, title=f'{selected_transaction} Linear Fit'),
        ).properties(
            width=800,  # Set chart width
            height=400,  # Set chart height
        )
        chart += linear_curve

    # Show the chart
    display(chart)

# Define an observer to update the plot when the widget values change:
widgets.interactive(update_plot,
                    selected_country=country_dropdown,
                    selected_transaction=transaction_dropdown,
                    start_year=start_year_slider,
                    end_year=end_year_slider,
                    show_linear_curve=show_linear_curve_radio)


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

' ### TESTING

In [ ]:
from sklearn.linear_model import LinearRegression

def add_linear_curve_to_dataframe(merged_df, selected_transaction):
    # Create the linear regression model
    model = LinearRegression()

    # Calculate the linear regression values and add them to the DataFrame
    X = merged_df['Year'].values.reshape(-1, 1)
    y = merged_df[selected_transaction].values.reshape(-1, 1)
    model.fit(X, y)
    linear_values = model.predict(X)

    # Add the linear regression values to the DataFrame
    merged_df[selected_transaction + '_Linear'] = linear_values.flatten()
    # merged_df[selected_transaction + '_Linear'] = linear_values.flatten().astype('float')

    return merged_df


#### WORKING

In [ ]:
# Import the necessary libraries:
import os
import pandas as pd
import ipywidgets as widgets
import altair as alt
from IPython.display import display
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

# Create a dropdown widget to select a country:
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')

# Create a dropdown widget to select a transaction:
transaction_dropdown = widgets.Dropdown(options=original_df['Transaction'].unique(), description='Transaction:', style={'description_width': 'initial', 'width': '500px'})

# Create a toggle button (Yes/No) to activate extra curves:
activate_extra_curves_toggle = widgets.ToggleButtons(options=['No', 'Yes'], description='Activate Extra Curves:', value='No')

# Create a radio button widget to toggle showing the linear or S-shaped curve:
curve_type_radio = widgets.RadioButtons(options=[('Linear', 'linear'), ('S-Shaped', 's_shaped')], description='Curve Type:', value='linear')

# Create widgets to select start_year and end_year:
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Modify the update_plot function to include the selected_transaction, start_year, end_year, curve_type, and activate_extra_curves arguments:
def update_plot(selected_country, selected_transaction, activate_extra_curves, curve_type, start_year, end_year):
    # Filter the DataFrame based on the selected country, transaction, and year range
    df = original_df[
        (original_df['Transaction'] == selected_transaction) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe_LocationYear(df)

    if curve_type == 'linear':
        # Create the linear regression model
        model = LinearRegression()
        X = merged_df['Year'].values.reshape(-1, 1)
        y = merged_df[selected_transaction].values.reshape(-1, 1)
        model.fit(X, y)
        linear_values = model.predict(X)
        # Add the linear regression values to the DataFrame
        merged_df[selected_transaction + '_linear'] = linear_values.flatten()

    elif curve_type == 's_shaped':
        # Fit the S-shaped curve to the data using the selected_transaction + '_linear' column as an estimation
        # Define the S-shaped curve function
        def s_curve(x, a, b, c):
            return a / (1 + np.exp(-b * (x - c)))
        if 'linear' not in merged_df.columns:
            # Calculate and add the linear curve
            model = LinearRegression()
            X = merged_df['Year'].values.reshape(-1, 1)
            y = merged_df[selected_transaction].values.reshape(-1, 1)
            model.fit(X, y)
            linear_values = model.predict(X)
            merged_df[selected_transaction + '_linear'] = linear_values.flatten()

        # Fit the S-shaped curve using the linear curve as an estimation
        linear_estimation = merged_df[selected_transaction + '_linear'].values
        popt, _ = curve_fit(s_curve, merged_df['Year'], linear_estimation)
        a, b, c = popt
        s_shaped_values = s_curve(merged_df['Year'], a, b, c)
        # Add the S-shaped curve values to the DataFrame
        merged_df[selected_transaction + '_s_shaped'] = s_shaped_values

    # Create the Altair chart for the selected transaction
    chart = alt.Chart(merged_df).mark_line(point=True).encode(
        x='Year:O',  # Year as ordinal (categorical)
        y=alt.Y(selected_transaction),  # selected transaction as quantitative
        color=alt.value('blue')  # Set color to blue
    ).properties(
        width=800,  # Set chart width
        height=400,  # Set chart height
        title=f'{selected_country} - {selected_transaction}'
    )

    # Create extra curves if the "Activate Extra Curves" toggle button is set to 'Yes'
    if activate_extra_curves == 'No':
        # Add code here to create and add extra curves to the chart
        print("")
    else:
        # Create the linear or S-shaped curve if the radio button is selected
        if curve_type == 'linear':
            linear_data = selected_transaction + '_linear:Q'
            linear_curve = alt.Chart(merged_df).mark_line(strokeDash=[10, 5], color='red').encode(
                x='Year:O',
                y=alt.Y(linear_data, title=f'{selected_transaction} Linear Fit'),
            )
            chart += linear_curve
        elif curve_type == 's_shaped':
            s_shaped_data = selected_transaction + '_s_shaped:Q'
            s_shaped_curve = alt.Chart(merged_df).mark_line(strokeDash=[10, 5], color='green').encode(
                x='Year:O',
                y=alt.Y(s_shaped_data, title=f'{selected_transaction} S-Shaped Fit'),
            )
            chart += s_shaped_curve

    # Show the chart
    display(chart)

# Define an observer to update the plot when the widget values change:
widgets.interactive(update_plot,
                    selected_country=country_dropdown,
                    selected_transaction=transaction_dropdown,
                    activate_extra_curves=activate_extra_curves_toggle,
                    curve_type=curve_type_radio,
                    start_year=start_year_slider,
                    end_year=end_year_slider
                    )


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

In [ ]:
!pip install vega

## Multiple choices

In [ ]:
import pandas as pd

def process_dataframe_LocationYear_ThisTransaction(df, ThisTransaction):
    print(ThisTransaction)
    # Filter the DataFrame based on the 'Transaction' column
    df_filtered = df[df['Transaction'] == ThisTransaction]
    # Group by 'LOCATION_Year' and aggregate values (e.g., using mean or sum)
    # df_pivot = df_filtered.groupby('LOCATION_Year').agg({'Value': 'sum'}).reset_index()
    #Pivot the DataFrame to separate "Transaction" into columns
    df_pivot = df_filtered.pivot_table(index='LOCATION_Year', columns='Transaction', values='Value', aggfunc='sum')
    # Create a new DataFrame 'df_new'
    selected_columns = ['LOCATION_Year', 'LOCATION', 'Country', 'Year']
    df_new = df[selected_columns].copy()  # Make a copy of the DataFrame
    df_new.drop_duplicates(subset='LOCATION_Year', keep='first', inplace=True)

    # Merge the DataFrames df_new and df_pivot on 'LOCATION_Year'
    merged_df = df_new.merge(df_pivot, on='LOCATION_Year')
    return merged_df

# Assuming you have a DataFrame named 'original_df'
unique_transactions = original_df['Transaction'].unique()

# Initialize an empty DataFrame to store the merged results
merged_df_base = pd.DataFrame()

# Process the first transaction and assign it to merged_df_base
merged_df_base = process_dataframe_LocationYear_ThisTransaction(original_df, unique_transactions[0])
# print(merged_df_base.head())

# Loop through unique transactions (starting from the second one)
for i in range(1, len(unique_transactions)):
    transaction = unique_transactions[i]
    df_result = process_dataframe_LocationYear_ThisTransaction(original_df, transaction)
    # Drop the specified columns from df_result
    columns_to_drop = ['LOCATION', 'Country', 'Year']
    df_result.drop(columns=columns_to_drop, inplace=True)
    # Check if there's any common data between df_result and merged_df_base
    common_data = merged_df_base.merge(df_result, on=['LOCATION_Year'], how='left', suffixes=('', f'_{transaction}'))
    if not common_data.empty:
        # Merge df_result with merged_df_base, specifying suffixes to avoid column conflicts
        merged_df_base = merged_df_base.merge(df_result, on=['LOCATION_Year'], how='left', suffixes=('', f'_{transaction}'))


merged_df_base['Inventories_Percentage_of_GDP'] = (merged_df_base['Changes in inventories'] / merged_df_base['Gross domestic product (expenditure approach)']) * 100
merged_df_base = merged_df_base.dropna(subset=['Inventories_Percentage_of_GDP'])# Assuming you want to sort merged_df_base by the 'LOCATION_Year' column in ascending order
merged_df_base = merged_df_base.sort_values(by='LOCATION_Year', ascending=True)

print(f"A total of {len(unique_transactions)} transactions processed.")


Gross domestic product (output approach)
Gross value added at basic prices, excluding FISIM
Gross value added at basic prices, total activity
Agriculture, hunting and forestry; fishing (ISIC rev3)
Exports of goods
Final consumption expenditure of households
Changes in inventories
Individual consumption expenditure of general government
of which: Manufacturing (ISIC rev3)
Imports of goods
Households and Non-profit institutions serving households
Domestic demand
Compensation of employees
Gross fixed capital formation
Taxes on production and imports
Exports of goods and services
of which: Manufacturing (ISIC rev4)
Industry, including energy (ISIC rev3)
Subsidies on production and imports
Imports of goods and services
Changes in inventories and acquisitions less disposals of valuables
Gross operating surplus and gross mixed income
Final consumption expenditure of general government
Final consumption expenditure
Imports of services
Collective consumption expenditure of general government
Ex

# Dashboard
a slider widget to select start_year and end_year,
a multi-choice widget to select countries, and
a multi-choice widget to select transactions.

The script is a Python code that creates a dashboard with interactive widgets to plot time-series data using the Pandas and Matplotlib libraries. The script imports the necessary libraries and defines a function to process the filtered DataFrame. The script then creates several widgets, including a slider widget to select start_year and end_year, a multi-choice widget to select countries, and a multi-choice widget to select transactions. The script also creates a plot output area and defines a function to update the plot based on the selected countries, transactions, and year range. The script creates an observer to update the plot when the selection or year range changes and displays the dashboard using the IPython.display library. The search results are not directly related to the script, but they provide additional information about using Pandas for data processing and analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output
from ipywidgets import Layout, Box


# Process the filtered DataFrame
# merged_df_base = process_dataframe_LocationYear(original_df)

# Create a dropdown widget to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Create a multi-choice widget to select countries
countries_options = sorted(merged_df_base['Country'].unique())
default_country = 'Germany' if 'Germany' in countries_options else countries_options[0]  # Set default to 'Germany' if available, else use the first option

# Create a multi-choice widget to select countries
selected_countries_selector = widgets.SelectMultiple(
    options=sorted(merged_df_base['Country'].unique()),  # Use merged_df_base instead of original_df
    # value=['Germany'],  # Default selected countries (can be empty)
    value=[default_country],  # Default selected country
    description='Select Countries:',
    rows=10  # Adjust the number of visible rows as needed
)

# Get a list of unique transactions from the columns of merged_df_base
unique_transactions = [col for col in merged_df_base.columns if col not in ['LOCATION_Year', 'LOCATION', 'Country', 'Year']]

# Create a multi-choice widget to select transactions
selected_transactions_selector = widgets.SelectMultiple(
    options=unique_transactions,
    value=[unique_transactions[0]],
    description='Select Transactions:',
    rows=10
)

# Create a plot output area
plot_output = widgets.Output()

# Define a function to update the plot based on the selected countries, transactions, and year range
def update_plot(selected_countries, selected_transactions, start_year, end_year):
    with plot_output:
        clear_output(wait=True)  # Clear previous plot
        # Filter the DataFrame based on the selected countries, transactions, and year range
        df_new = merged_df_base[
            (merged_df_base['Country'].isin(selected_countries)) &
            # (merged_df_base['Transaction'].isin(selected_transactions)) &
            (merged_df_base['Year'] >= start_year) &
            (merged_df_base['Year'] <= end_year)
        ]

        # Process the filtered DataFrame
        # plt.figure(figsize=(13, 6))
        # for transaction in selected_transactions:
        #     plt.title(f'{transaction} per Year')
        #     plt.xlabel('Year')
        #     plt.ylabel(transaction)
        #     for country in selected_countries:
        #         country_df = df_new[(df_new['Country'] == country) ]#& (df_new['Transaction'] == transaction)]
        #         plt.plot(country_df['Year'], country_df[transaction], marker='o', label=f'{country} - {transaction}')
        #     plt.legend(title='Countries')
        #     plt.grid(True)  # Add grid lines to the plot
        #     plt.show()

        # Process the filtered DataFrame
        fig, ax = plt.subplots(figsize=(10, 6))  # Specify the figsize for all plots
        for transaction in selected_transactions:
            ax.set_title(f'{transaction} per Year')
            ax.set_xlabel('Year')
            ax.set_ylabel(transaction)
            for country in selected_countries:
                country_df = df_new[(df_new['Country'] == country) ]#& (df_new['Transaction'] == transaction)]
                ax.plot(country_df['Year'], country_df[transaction], marker='o', label=f'{country} - {transaction}')
            ax.legend(title='Countries')
            ax.grid(True)  # Add grid lines to the plot
        plt.show()

# Create an observer to update the plot when the selection or year range changes
controls = widgets.VBox([start_year_slider, end_year_slider, selected_countries_selector, selected_transactions_selector])
box_layout = Layout(display='flex', flex_flow='row wrap', justify_content='space-between', width='100%')
app_layout = Box(children=[controls, plot_output], layout=box_layout)
widgets.interactive(update_plot, selected_countries=selected_countries_selector,
                    selected_transactions=selected_transactions_selector,
                    start_year=start_year_slider, end_year=end_year_slider)

display(app_layout)


Box(children=(VBox(children=(IntSlider(value=1990, description='Start Year:', max=2017, min=1970), IntSlider(v…

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output
from ipywidgets import Layout, Box


# Process the filtered DataFrame
# merged_df_base = process_dataframe_LocationYear(original_df)

# Create a dropdown widget to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Create a multi-choice widget to select countries
countries_options = sorted(merged_df_base['Country'].unique())
default_country = 'Germany' if 'Germany' in countries_options else countries_options[0]  # Set default to 'Germany' if available, else use the first option

# Create a multi-choice widget to select countries
selected_countries_selector = widgets.SelectMultiple(
    options=sorted(merged_df_base['Country'].unique()),  # Use merged_df_base instead of original_df
    # value=['Germany'],  # Default selected countries (can be empty)
    value=[default_country],  # Default selected country
    description='Select Countries:',
    rows=10  # Adjust the number of visible rows as needed
)

# Get a list of unique transactions from the columns of merged_df_base
unique_transactions = [col for col in merged_df_base.columns if col not in ['LOCATION_Year', 'LOCATION', 'Country', 'Year']]

# Create a multi-choice widget to select transactions
selected_transactions_selector = widgets.SelectMultiple(
    options=unique_transactions,
    value=[unique_transactions[0]],
    description='Select Transactions:',
    rows=10
)

# Create a plot output area
plot_output = widgets.Output()

# Define a function to update the plot based on the selected countries, transactions, and year range
def update_plot(selected_countries, selected_transactions, start_year, end_year):
    with plot_output:
        clear_output(wait=True)  # Clear previous plot
        # Filter the DataFrame based on the selected countries, transactions, and year range
        df_new = merged_df_base[
            (merged_df_base['Country'].isin(selected_countries)) &
            # (merged_df_base['Transaction'].isin(selected_transactions)) &
            (merged_df_base['Year'] >= start_year) &
            (merged_df_base['Year'] <= end_year)
        ]

        # Process the filtered DataFrame
        plt.figure(figsize=(13, 6))
        for transaction in selected_transactions:
            plt.title(f'{transaction} per Year')
            plt.xlabel('Year')
            plt.ylabel(transaction)
            for country in selected_countries:
                country_df = df_new[(df_new['Country'] == country) ]#& (df_new['Transaction'] == transaction)]
                plt.plot(country_df['Year'], country_df[transaction], marker='o', label=f'{country} - {transaction}')
            plt.legend(title='Countries')
            plt.grid(True)  # Add grid lines to the plot
            plt.show()

        # # Process the filtered DataFrame
        # fig, ax = plt.subplots(figsize=(10, 6))  # Specify the figsize for all plots
        # for transaction in selected_transactions:
        #     ax.set_title(f'{transaction} per Year')
        #     ax.set_xlabel('Year')
        #     ax.set_ylabel(transaction)
        #     for country in selected_countries:
        #         country_df = df_new[(df_new['Country'] == country) ]#& (df_new['Transaction'] == transaction)]
        #         ax.plot(country_df['Year'], country_df[transaction], marker='o', label=f'{country} - {transaction}')
        #     ax.legend(title='Countries')
        #     ax.grid(True)  # Add grid lines to the plot
        # plt.show()

# Create an observer to update the plot when the selection or year range changes
controls = widgets.VBox([start_year_slider, end_year_slider, selected_countries_selector, selected_transactions_selector])
box_layout = Layout(display='flex', flex_flow='row wrap', justify_content='space-between', width='100%')
app_layout = Box(children=[controls, plot_output], layout=box_layout)
widgets.interactive(update_plot, selected_countries=selected_countries_selector,
                    selected_transactions=selected_transactions_selector,
                    start_year=start_year_slider, end_year=end_year_slider)

display(app_layout)


Box(children=(VBox(children=(IntSlider(value=1990, description='Start Year:', max=2017, min=1970), IntSlider(v…